# Greeting

หลายคนที่เป็น Data Scientist มักจะไม่ค่อยใช้ Design pattern มากเท่าไหร่ เพราะ การทำ Design pattern ดูเหมือนจะเรื่องของ Software engineer และจำเป็นต้องมีความเข้าใจเรื่อง OOP เลยทำให้เหมือนมีความซับซ้อนระดับหนึ่ง
<br/><br/>
แต่จากประสบการณ์การเป็น Data Scientist เกือบสองปี การเสียเวลาทำ Design pattern สักนิด ไม่เพียงแต่จะให้ code ของคุณดูโปรมากขึ้นแล้ว แต่ยังเป็นการลดความขัดแย้งกันระหว่างทีมอื่น เช่น Python devoloper, Data engineer, DevOps หรือ แม้แต่ตัวคุณเองใ่นอนาคตด้วย
<br/><br/>
และขอบอกกง ๆ OOP ใน Python มีความเมตตาปราณีมากกว่าภาษาอื่นเยอะเลยนะครับ
<br/><br/>
ซั่งใน blog นี้จะสาธิตวิธีการใช้ design pattern ที่ชื่อว่า Adapter มาจัดการไฟล์ Crystallographic Information Framework (CIF) จากการแข่ง TMLCC กันครับ

<center>
<img src="/work/assets/images/TMLCC_Logo.4bbf95cb.png"></img>
รูปจาก https://tmlcc.cseathai.org/
<center>


# Adapter

<center>
<img src="/work/assets/images/adapter-mini-2x.png"></img>
รูปจาก https://refactoring.guru/design-patterns/python
<center>


## Start with problem

<center>
<img src="/work/assets/images/problem-en-2x.png"></img>
</center>

เว็บไซต์ refactoring.guru ได้บอกถึงปัญหาของการเขียน code ว่า เรามีได้เขียน `Analyics Library` เพื่อประมวณผลราคาหุ้น ซึ่งจะรับข้อมูลใน format ของ JSON ซึ่งเป็น format ที่โบร๊คเกอร์ส่วนใหญ่ใช้กัน วันดีคืนดีดันมีลูกค้ามาขอใช้บริการ `Analyics Library` ของเราด้วย แ่ต่ API ที่ทาง broker เจ้าใหม่ ดันส่งข้อมูลที่มี format เป็น XML ซึ่งไม่เข้ากับ library เรา
<br/><br/>
ดังันั้นสิ่งที่เราต้องทำต้องทำ library ของเรา รองรับกับ data structure ของลูกค้าคนเจ้าใหม่
<br/><br/>
***ปัญหา คือ***

เราควรแก้ code ที่ส่วนไหนดี ถ้าแก้ที่ `Analyics Library` ก็อาจจะไปกระทบกับูกค้าเจ้าอื่น และถ้าโตรงสร้างของ XML มีการเปลี่ยนแปลงในอนาคต การแก้ก็จะทำได้ยาก ลองนึกภาพว่า `Analyics Library` มี code ยั่วเยี้ยนไปหมด ถ้าจะแก้ทีคงไม่รู้จะแก้จกากจุดไหน

## End with solution!

<center>
<img src="/work/assets/images/solution-en-2x.png"></img>
</center>

ทางเว็บ refactoring.guru ได้แนะนำวิธีการแก้ปัญหา เร่าจะเขียน class ที่มีชื่อว่า `Adapter` เพื่อมาเป็นตัวกลางในการแปลงข้อมูลระหว่างสองส่วนนี้
<br/><br/>
วิธีจะไม่ทำให้กระทบ service หลักของเรา และถ้าโครงสร้างข้อมูลมีการเปลี่ยนแปลงในอนาคต เราก็แค่มาแก้ที่จุดเดียวไปเลย

## Apply to CIF

<center><img src="/work/assets/images/Screenshot 2564-10-01 at 00.59.53.png"></img></center>

Crystallographic Information Framework เป็นโครงสร้างข้อมูลของโมเลกุล ที่ถูกออกแบบ software สามารถประมวลผล และแสดงผล รูปร่างของผลึกทางเคมีของโมเลกุลนั้น ๆ ได้ โดยข้อมูลจะมีลักษณะเป็นโครงสร้างคล้าย YAML หรือ XML
<br/><br/>
ในการทำ adapter เราจะแปลงให้ CIF ไปดูในรูปของ Pandas DataFrame object บน Python เพื่อให้ง่ายต่อการคำนวณ

# Let's code!

## Import modules

In [1]:
# OS command
import os

# Sure, we need `pandas`
import pandas as pd

# Typing
from typing import (
    List,
    Tuple
)

# Abstract classes we're going to use them  in the section
from abc import ABC, abstractmethod

# Base model
from pydantic import BaseModel

## Abstract Adapter

In [2]:
class AbstractAdapter(ABC):
    @abstractmethod
    def extract(self):
        """
        Extract input data
        """

    def tranform(self):
        """
        Tranformed data into desired format
        """

    def load(self):
        """
        Load data to next step
        """

    @abstractmethod
    def apply(self):
        """
        Apply data ETL process
        """
        pass


Abstract class ข้างบนจะบอก adapter process โดยภาพรวมนะครับ โดยผมเขียนให้อยู่ในรูปของ ETL เพื่อให้เข้ากับหลัก Data engineer

## CIF Adapter

### Adapter desired output

In [23]:
class CIF2PandasAdapterOutput(BaseModel):
    """
    Set up desired output
    """
    
    metadata: pd.DataFrame
    loops: List[pd.DataFrame]

    class Config:
        """
        Set some config to allow pd.DataFrame to BaseModel
        """
        arbitrary_types_allowed = True

### Set up adapter

In [24]:
class CIF2PandasAdapter:
    """
    An adapter to convert CIF into Pandas DataFrames
    """

    @staticmethod
    def load_cif(cif_filepath: str) -> List[str]:
        """
        Read CIF file as String and split looping sections
        """
        with open(cif_filepath) as f:
            filename = f.readline().strip()
            dataframes = []
            dataframe = []
            for line in f.readlines():
                columns = [
                    l.strip() for l in line.strip().split(" ") if l and (l != "")
                ]
                if columns:
                    if columns[0] == "loop_":
                        dataframes.append(dataframe)
                        dataframe = []
                    else:
                        if "fapswitch" not in columns and columns != [""]:
                            dataframe.append(columns)
            dataframes.append(dataframe)

        return dataframes

    @staticmethod
    def get_metadata(dataframes: List[List[str]]) -> pd.DataFrame:
        """
        Get metadata of a CIF file
        """
        return pd.DataFrame(dataframes[0])

    @staticmethod
    def get_loops(dataframes: List[List[str]]) -> List[pd.DataFrame]:
        """
        Get loops
        """
        loops = []
        for dataframe in dataframes[1:]:
            loop = pd.DataFrame(dataframe)
            loop_fixed = loop[loop[1].notna()]
            loop_fixed.columns = loop[loop[1].isna()][0]
            loops.append(loop_fixed)

        return loops

    def extract(self, cif_filepath: str) -> List[str]:
        """
        Read filepath and return sectioned string
        """
        return self.load_cif(cif_filepath)

    def transform(self, cif_list: List[str]) -> Tuple[pd.DataFrame, List[pd.DataFrame]]:
        """
        Transform CIF text into datafram
        :params:
        cif_list: List of sectioned CIF text
        :return:
        Tuple of :
        (
            dataframe of CIF metadata,
            Tuple of (
                dataframe of loop_1,
                dataframe of loop_2,
            )
        )
        """
        metadata = self.get_metadata(cif_list)
        extract_loops = self.get_loops(cif_list)
        return metadata, extract_loops

    def load(
        self, metadata: pd.DataFrame, extract_loops: List[pd.DataFrame]
    ) -> CIF2PandasAdapterOutput:
        """
        Load tranformed data into desired output
        """
        output = CIF2PandasAdapterOutput(metadata=metadata, loops=extract_loops)
        return output

    def apply(self, cif_filepath: str) -> List[pd.DataFrame]:
        """
        Apply ETL pipeline
        """
        # Extract
        cif_list = self.extract(cif_filepath)

        # Transform
        metadata = self.get_metadata(cif_list)
        extract_loops = self.get_loops(cif_list)

        # Load
        output = self.load(metadata, extract_loops)

        return output

### Let's test our adapter

คราวนี้เราลองมาทดสอบตัว adapter ที่เราเขียนนะครับ

In [28]:
# สร้าง instance
adapter = CIF2PandasAdapter()

# Apply adapter
output = adapter.apply("/work/assets/cif/mof_unit_pretest_1606.cif")

In [29]:
output.metadata

,0,1
0,_audit_creation_date,2012-12-11T16:15:10-0500
1,_symmetry_space_group_name_H-M,P1
2,_symmetry_Int_Tables_number,1
3,_space_group_crystal_system,triclinic
4,_cell_length_a,8.228802
5,_cell_length_b,14.951691
6,_cell_length_c,17.354118
7,_cell_angle_alpha,96.677539
8,_cell_angle_beta,95.887047
9,_cell_angle_gamma,97.930399


In [27]:
output.loops[0]

,_atom_site_label,_atom_site_type_symbol,_atom_type_description,_atom_site_fract_x,_atom_site_fract_y,_atom_site_fract_z,_atom_type_partial_charge
7,Ba1,Ba,Ba6+2,0.159600,0.544390,0.464780,1.802869
8,Ba2,Ba,Ba6+2,0.650580,0.455600,0.535150,1.731705
9,C1,C,C_3,0.371183,0.776470,0.179085,0.469523
10,C2,C,C_R,0.888781,0.792702,0.520630,0.031324
11,C3,C,C_R,0.070499,0.235341,0.526809,-0.395251
...,...,...,...,...,...,...,...
132,P4,P,P_3+3,0.815490,0.291960,0.432360,1.021380
133,P5,P,P_3+3,0.879666,0.377398,0.735117,1.058954
134,P6,P,P_3+3,0.426042,0.785100,0.408644,1.084319
135,P7,P,P_3+3,0.474452,0.573225,0.690302,0.972787


In [30]:
output.loops[1]

,_geom_bond_atom_site_label_1,_geom_bond_atom_site_label_2,_ccdc_geom_bond_type
3,O2,P5,D
4,C42,H11,S
5,C14,H17,S
6,C34,H4,S
7,Ba1,O1,S
...,...,...,...
149,C20,C21,A
150,O20,P4,S
151,C36,O13,S
152,Ba1,O4,S


# Reference

* https://refactoring.guru/design-patterns/python
* https://tmlcc.cseathai.org/
* https://www.iucr.org/resources/cif
* https://www.geeksforgeeks.org/abstract-classes-in-python/

# About me

ชื่อ เปรม aka `batprem` จบ Data Analytics จาก University of Sheffield ประเทศอังกฤษ ปัจจุบันเป็น Full stack data scientist ทำตั้งแต่สากกระเบือยันเรือรบ scrape ช้อมูล, ทำ pipeline, machine learning model
<br/><br/>
ปัจจุบันเป็นเจ้าของเพจ [ข้อมูลกระตุกจิตกระช่ากใจเพื่อบรรลุ data scientist](https://www.facebook.com/Deangdata).


<a href="https://www.linkedin.com/in/premchote">
    <img style="float: left" alt="linkedIn" src="https://static-exp1.licdn.com/sc/h/al2o9zrvru7aqj8e1x2rzsrca"
    width=50" height="50"></img>
</a>
<a href="https://www.kaggle.com/batprem" class="inline-block">
    <img style="float: left" alt="kaggle" src="https://www.kaggle.com/static/images/favicon.ico"
    width=50" height="50"></img>
</a>
<a href="https://www.credential.net/fbb28b4b-f61a-4d21-9b96-5b7fba6b83cf#gs.cnor4e" class="inline-block">
    <img style="float: left" alt="kaggle" src="https://s3.us-east-1.amazonaws.com/accredible-api-templates/15784284048332915386973343827272.png"
    width=50" height="50"></img>
</a>
<br/><br/><br/><br/>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a340163b-e04e-42b4-96b8-0f3c9760808a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>